In [ ]:
!pip install langchain
!pip install langchain_chroma
!pip install sentence_transformers
!pip install aiofiles
!pip install accelerate
!pip install pyPDF
!pip install tiktoken
!pip install einops
!pip install nltk


In [1]:
from langchain_community.document_loaders import PyPDFLoader as pdf

loader = pdf("book.pdf")
pages = loader.load()
print(pages)

text=" "
for i in range(0,len(pages)):
  text=text+pages[i].page_content


[Document(page_content=' \nADNOC -CICV -505A(v 6)    \n ADNOC Classification: Need -To-Know  \n_____________________________________________  \nEPC AGREEMENT  \nFOR  \nHabshan 3 Stabilizer Feed Capacity Enhancement & Other Mechanical Works  \n(FOR ONSHORE WORKS)  \nREFERENCE NUMBER [●] \n_____________________________________________  \n ', metadata={'source': 'book.pdf', 'page': 0}), Document(page_content=' \nADNOC -CICV -505A(v 6) 1 insert the reference numbe r \n ADNOC Classification: Need -To-Know  \nFORM OF AGREEMENT  \nThis AGREEMENT is made on the date of the later of the two signatures below, by and between:  \n(1) [ADNOC Gas ], a company established under the laws of ABU DHABI, and having its registered office \naddress at [insert registered address ], Abu Dhabi, United Arab Emirates (the “COMPANY ”); and  \n(2) [INSERT LEGAL NAME OF THE CONTRACTOR ], a company organised and existing under the laws \nof [insert jurisdiction ], with commercial licence no. [●] / company number [●

In [ ]:
# # Open the file in read mode
# with open('new_documents.txt', 'r') as file:
#     # Read the contents of the file into a string
#     new_documents = file.read()

# # Now the variable 'text' contains the contents of the file as a string
# print(new_documents)


In [ ]:
# part_length = len(new_documents) // 20

# chunks = []

# for i in range(0, len(new_documents), part_length):
#     chunks.append(new_documents[i:i+part_length])

# print(chunks)

In [ ]:
# part_length = len(new_documents1) // 20

# chunks1 = []
# for i in range(0, len(new_documents1), part_length):
#     chunks1.append(new_documents1[i:i+part_length])

# # Print the parts to verify
# print(chunks1)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from accelerate import Accelerator

# Initialize Accelerator
accelerator = Accelerator()
torch.random.manual_seed(0)

def start_model():
    model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
    )
    tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

    pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    )
    return model,tokenizer ,pipe

model,tokenizer,pipe=start_model()


In [4]:
#Text Splitter and Embeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters import CharacterTextSplitter
import re
import langchain
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain_text_splitters import SpacyTextSplitter


def text_splitter_And_Similarity_Search(documents):



    model_name="BAAI/bge-large-en-v1.5"
    model_kwargs = {"device": "cpu","trust_remote_code":True}
    encode_kwargs = {"normalize_embeddings": True}

    # text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
    #                                                    chunk_overlap=50)




    embedding_function = SentenceTransformerEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    )

    db = Chroma.from_documents(documents, embedding_function )


    # embedding_function = SentenceTransformerEmbeddings(model_name="thenlper/gte-base")
    # db = Chroma.from_documents(documents, embedding_function )

    return db







In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

docs=Document(page_content=text,metadata={"source":"book"})
doc=[]
doc.append(docs)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                                       chunk_overlap=50)

documents = text_splitter.split_documents(doc)

In [6]:
db=text_splitter_And_Similarity_Search(documents)

In [7]:
k=20
def search(prompt):
  docs = db.similarity_search_with_score(prompt,k=k)
  return docs

In [8]:
#Generation
prompt="What is CONTRACTOR limit of liability?"
docs=search(prompt)
new_documents=" "
for i in range(0,k):
    new_documents=new_documents+docs[i][0].page_content





In [21]:
prompt1="Under what condition COMPANY can terminate the contract?"
docs1=search(prompt1)
new_documents1=""
for i in range(0,k):
    new_documents1=new_documents1+docs1[i][0].page_content



In [9]:

generation_args = {
 "max_new_tokens": 4000,
 "return_full_text": False,
 "repetition_penalty": 1.1,
 "temperature": 0.0,
 "do_sample": False
}

def get_summary_prompt(text):

    messages = f"""
      You are given a text , Your task is to summarize the text and make it shorted than its orignal size.
    \n
        text : {text}

    """
    return messages



In [10]:
def get_generate_prompt(query, retrieved_documents):
    messages = f"""Answer the quesion Accuratrly and precisely : {query} , using the  information given below Donot repeat anything in the answer .

        Information : {retrieved_documents}."
    """
    return messages


In [11]:
def model_summarize(messages):
    output = pipe(messages, **generation_args)
    return output

In [12]:
import time

summary11=""

for i in range(0,k):
  sum = model_summarize(get_summary_prompt(docs[i][0].page_content))
  print(sum[0])
  summary11=summary11+sum[0]["generated_text"]




/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


{'generated_text': "\nSummary: Liability for gross negligence/wilful misconduct, breaches of specific articles, third-party claims, and certain indemnities are excluded from limitation clauses. Contractor's warranty of title and payment responsibilities also have specified exceptions.\n\n**Response:** Limitation of liability caps gross negligence/misconduct, but excludes such cases, breaches of Articles 37, 33, and related indemnities. Exceptions include third-party claims and contractor's title warranties & payment duties."}
{'generated_text': '\n    Summary : Contractor must compensate Company for property losses due to contract performance issues. Maximum coverage limit is $1 million per incident.\n\n- [Response]: Contractor indemnifies Company against property losses related to contract execution, capped at $1M per incident.'}
{'generated_text': "\n    Summary: The agreement allows the company to suspend or terminate work due to force majeure events lasting over 60 days continuousl

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


{'generated_text': '\n    Summary: Contractor must fulfill obligations stated in Articles 22.1(a)-22.1(e)(i) & 22.1(f); Company will be protected if contractor fails to comply.\n\n- [Response]: Contractor agrees to adhere to obligations in Articles 22.1(a)-22.1(e)(i) & 22.1(f); company receives protection for non-compliance.'}
{'generated_text': "\nSummary: The contractor must indemnify the company for any third-party losses due to death, illness, injury, property damage, or pollution caused by their work on the project.\n\nResponse=The contractor is obligated to compensate the company for third-party harm resulting from death, illness, injuries, property damage, or pollution linked to the contractor's project activities. This includes both direct consequences at the contractor's site and indirect effects like pollution emanating from the contractor's property."}
{'generated_text': "\n    Summary: The contractor must restore company information promptly after being notified, within fiv

In [ ]:
len(new_documents1)

19178

In [22]:
import time

summary12=""

for i in range(0,k):
  sum1 = model_summarize(get_summary_prompt(docs1[i][0].page_content))
  print(sum1[0])
  summary12=summary12+sum1[0]["generated_text"]




{'generated_text': '\n    Summary : The contractor can terminate the agreement under certain conditions outlined in Article 23.7(a)(ii)(B). However, termination cannot occur due to any reasons unless specified in the agreement. Additionally, specific articles from the Civil Code are excluded from applicability.\n\n[Response]: Contractor can terminate agreement per Article 23.7(a)(ii)(B), but only under stated conditions; no grounds for termination outside this clause allowed, with exclusions from Civil Code provisions.'}
{'generated_text': "\n    Summary: If the agreement gets suspended partially or fully, parties remain responsible for their obligations. Company can end the contract with 30 days' notice but must finish remaining works or hire others to do so.\n\n- [Response]: Upon partial/full agreement suspension, parties retain obligations; company can terminate with 30-day notice, requiring completion of pending tasks or third-party involvement."}
{'generated_text': "\n    Summary:

In [23]:
summary11=summary11.replace("[response]:","")
summary11=summary11.replace("[Response]:","")
summary11=summary11.replace("summary=","")
summary11=summary11.replace("response=","")
summary11=summary11.replace("Response=","")
summary11=summary11.replace("response:","")
summary11=summary11.replace("Response:","")
summary11=summary11.replace("output:","")
summary11=summary11.replace("Output:","")
summary11=summary11.replace("Summary:","")
summary11=summary11.replace("Summary :","")
summary11=summary11.replace("Expected Output:","")
summary11=summary11.replace("Expected:","")



summary12=summary12.replace("[response]:","")
summary12=summary12.replace("[Response]:","")
summary12=summary12.replace("summary=","")
summary12=summary12.replace("response=","")
summary12=summary12.replace("Response=","")
summary12=summary12.replace("output:","")
summary12=summary12.replace("Output:","")
summary12=summary12.replace("output:","")
summary12=summary12.replace("Output:","")
summary12=summary12.replace("Summary:","")
summary12=summary12.replace("Summary :","")
summary12=summary12.replace("Expected Output:","")
summary12=summary12.replace("Expected:","")



In [15]:
summary11

"\n Liability for gross negligence/wilful misconduct, breaches of specific articles, third-party claims, and certain indemnities are excluded from limitation clauses. Contractor's warranty of title and payment responsibilities also have specified exceptions.\n\n**** Limitation of liability caps gross negligence/misconduct, but excludes such cases, breaches of Articles 37, 33, and related indemnities. Exceptions include third-party claims and contractor's title warranties & payment duties.\n     Contractor must compensate Company for property losses due to contract performance issues. Maximum coverage limit is $1 million per incident.\n\n-  Contractor indemnifies Company against property losses related to contract execution, capped at $1M per incident.\n     The agreement allows the company to suspend or terminate work due to force majeure events lasting over 60 days continuously or 180 days in total. During a suspension, the company can still terminate the contract according to Article

In [18]:
input_prompt = get_generate_prompt(prompt,summary11)

In [24]:
input_prompt1 = get_generate_prompt(prompt1,summary12)

In [25]:
len(input_prompt1)

9378

In [ ]:
new_documents1

In [ ]:
summary12

In [19]:
result = model_summarize(input_prompt)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [26]:
result1 = model_summarize(input_prompt1)


In [20]:
result

[{'generated_text': " Contractor must inform the company immediately upon discovering any hazardous substances present on the site. Failure to disclose may result in penalties as stipulated in the agreement.\n\n-  Contractor must alert the company of any hazardous substances found onsite; nondisclosure could lead to penalties.\n\n\n### Response\n\nContractor's maximum liability cap for property damage, personal injury, and other related claims is set at $1 million per incident, subject to limitations regarding gross negligence, wilful misconduct, and specific article breaches.\n\n\n**Key Points:**\n\n- **Liability Cap:** The contractor's liability for property damage, personal injury, and related claims is capped at $1 million per incident.\n\n- **Exclusions:** Claims arising from gross negligence, wilful misconduct, and breaches of specific articles (Articles 37, 33, and related indemnities) are not covered by this cap.\n\n- **Indemnification Obligations:** The contractor is obliged t

In [28]:
result1

[{'generated_text': " Contractor prohibited from transferring contractual duties/rights sans Company's express permission. Any unauthorized transfers lead to instant contract cessation.\n     The contractor is barred from delegating responsibilities/rights absent explicit authorization from the company. Unauthorized delegation leads to immediate contract termination.\n     Assignment/novation/transfer of contractual duties/rights without company's written consent constitutes a breach, triggering immediate termination.\n\n**Response:** Transfer of contractual duties/rights without company's written consent is forbidden and prompts immediate contract termination.\n     No assignment/novation/transfer of contractual obligations permitted without company's express written consent; unauthorized action causes immediate termination.\n     Prohibition against contractor transferring duties/rights without company's written consent; unauthorized transfer results in immediate termination.\n     B

In [ ]:
import gc
gc.collect()

450

In [32]:
def count_tokens(sentence):
  encoded_sentence = tokenizer.encode(sentence)
  token_ids = encoded_sentence[1]

  num_tokens =encoded_sentence

  return len(num_tokens)



tokens=count_tokens(result1[0]["generated_text"])
print(tokens)

483


In [ ]:
summary_length = len(summary12) // 20

chunks = []

for i in range(0, len(summary12), summary_length):
    chunks.append(summary12[i:i+summary_length])

print(chunks)

In [30]:
result

[{'generated_text': " Contractor must inform the company immediately upon discovering any hazardous substances present on the site. Failure to disclose may result in penalties as stipulated in the agreement.\n\n-  Contractor must alert the company of any hazardous substances found onsite; nondisclosure could lead to penalties.\n\n\n### Response\n\nContractor's maximum liability cap for property damage, personal injury, and other related claims is set at $1 million per incident, subject to limitations regarding gross negligence, wilful misconduct, and specific article breaches.\n\n\n**Key Points:**\n\n- **Liability Cap:** The contractor's liability for property damage, personal injury, and related claims is capped at $1 million per incident.\n\n- **Exclusions:** Claims arising from gross negligence, wilful misconduct, and breaches of specific articles (Articles 37, 33, and related indemnities) are not covered by this cap.\n\n- **Indemnification Obligations:** The contractor is obliged t